In [2]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import os
from catboost import CatBoostRegressor, Pool
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from gensim.models import Word2Vec
import time

In [3]:
!pip install --upgrade pip
!pip install tqdm
!pip install gensim

In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
train2018 = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/train_水压数据_2018.csv')
train2019 = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/train_水压数据_2019.csv')
train2020 = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/test_水压数据_2020.csv')
test = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/to_predict.csv')
submit = pd.read_csv('/Users/liruizhi/Downloads/实习/浪潮Inspur/数据文件/管网压力预测-数据/submit.csv')

In [ ]:
train2018.head()

In [6]:
train2020.head()

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2020-01-01,站点4,0.417375,0.431625,0.437625,0.439500,0.447375,0.445125,0.436500,0.422250,...,0.337875,0.336000,0.339750,0.327000,0.320250,0.330750,0.336375,0.326625,0.350625,0.386625
1,2020-01-01,站点7,0.292375,0.321250,0.327250,0.324625,0.331750,0.316750,0.309250,0.298750,...,0.268000,0.267250,0.264625,0.261625,0.250375,0.256375,0.255625,0.237250,0.257875,0.291625
2,2020-01-01,站点22,0.255250,0.307000,0.327625,0.323500,0.343000,0.314500,0.304000,0.259375,...,0.231625,0.226375,0.228250,0.229000,0.230500,0.226375,0.229375,0.226000,0.229750,0.239500
3,2020-01-01,站点21,0.292875,0.313125,0.321000,0.314625,0.321375,0.318375,0.303375,0.288375,...,0.263250,0.257250,0.265125,0.253875,0.247500,0.258000,0.247875,0.237750,0.252000,0.286125
4,2020-01-01,站点20,0.288375,0.316500,0.320250,0.314250,0.322500,0.318000,0.300750,0.283875,...,0.264000,0.256875,0.261750,0.258375,0.253125,0.259125,0.252000,0.240000,0.256125,0.285375


In [7]:
test.head()

,id,Time,MeasName,Hour
0,0,2020-02-03,站点4,H0
1,1,2020-02-03,站点4,H1
2,2,2020-02-03,站点4,H2
3,3,2020-02-03,站点4,H3
4,4,2020-02-03,站点4,H4


In [8]:
submit.head()

,id,pressure
0,0,0.4335
1,1,0.4335
2,2,0.4335
3,3,0.4335
4,4,0.4335


In [9]:
def reshape_data(df1):
    time = df1["Time"].values
    meas = df1["MeasName"].values

    df_list = []

    for i in range(0,24):
        hour = "H"+str(i)
        pressure = df1[hour].values
        df2 = pd.DataFrame()
        df2["Time"] = time
        df2["MeasName"] = meas
        df2["Hour"] = hour
        df2["pressure"] = pressure

        df_list.append(df2)

    df3 = pd.concat(df_list)
    df3.sort_values(by = ['Time', 'MeasName'], inplace = True)
    df3 = df3.reset_index(drop=True)
    return df3

In [10]:
train2018 = reshape_data(train2018)
train2019 = reshape_data(train2019)
train2020 = reshape_data(train2020)

In [11]:
train2018.head()

,Time,MeasName,Hour,pressure
0,2018-01-01,站点1,H0,0.288625
1,2018-01-01,站点1,H1,0.292000
2,2018-01-01,站点1,H2,0.290500
3,2018-01-01,站点1,H3,0.299500
4,2018-01-01,站点1,H4,0.300250


In [12]:
train2018['Time_time'] = pd.to_datetime(train2018['Time'])
train2019['Time_time'] = pd.to_datetime(train2019['Time'])
train2020['Time_time'] = pd.to_datetime(train2020['Time'])
test['Time_time'] = pd.to_datetime(test['Time'])

In [13]:
def abnormal(df):
    '''
    处理-9999异常值: 上一个值填充
    '''
    index_value = list(df[df['pressure'] == - 99999].index)
    for i in index_value:
        value = df[df.index == (i - 1)]['pressure'].iloc[0]
        df.loc[i, 'pressure'] = value
    return df
train2018 = abnormal(train2018)
train2019 = abnormal(train2019)
train2020 = abnormal(train2020)

In [14]:
def feature1(df):
    df['Day'] = df['Time'].apply(lambda x: int(x.split('-')[-1]))
    df['Hour'] = df['Hour'].apply(lambda x: int(x.replace('H', '')))
    df['MeasName'] = df['MeasName'].apply(lambda x: int(x.replace('站点', '')))
    
    return df

In [15]:
train2018 = feature1(train2018)
train2019 = feature1(train2019)
train2020 = feature1(train2020)
test = feature1(test)

In [17]:
train2020.head()

,Time,MeasName,Hour,pressure,Time_time,Day
0,2020-01-01,1,0,0.309625,2020-01-01,1
1,2020-01-01,1,1,0.325750,2020-01-01,1
2,2020-01-01,1,2,0.332875,2020-01-01,1
3,2020-01-01,1,3,0.324250,2020-01-01,1
4,2020-01-01,1,4,0.331750,2020-01-01,1


In [19]:
train2018.head()

,Time,MeasName,Hour,pressure,Time_time,Day
0,2018-01-01,1,0,0.288625,2018-01-01,1
1,2018-01-01,1,1,0.292000,2018-01-01,1
2,2018-01-01,1,2,0.290500,2018-01-01,1
3,2018-01-01,1,3,0.299500,2018-01-01,1
4,2018-01-01,1,4,0.300250,2018-01-01,1


In [20]:
train2019Mon2 = train2019[(train2019['Time_time'] >= '2019-2-1') & (train2019['Time_time'] <= '2019-2-28')] # 2019年2月数据
train2019Mon1 = train2019[(train2019['Time_time'] >= '2019-1-1') & (train2019['Time_time'] <= '2019-1-28')] # 2019年1月数据
Mon_2_1_2019 = train2019Mon2['pressure'].mean() - train2019Mon1['pressure'].mean() # 2月水压均值 - 1月水压均值

train1 = train2020[(train2020['Time_time'] >= '2020-1-1') & (train2020['Time_time'] <= '2020-1-31')] # 2020年1月数据
test1 = test[(test['Time_time'] >= '2020-2-3') & (test['Time_time'] <= '2020-2-16')] # 2020年2.3 - 2.16的数据


used_feat = [f for f in train1.columns if f not in ['id', 'pressure', 'Time', 'Time_time']]
print('feat nums ', len(used_feat), used_feat)

feat nums  3 ['MeasName', 'Hour', 'Day']


In [21]:
train1.head()

,Time,MeasName,Hour,pressure,Time_time,Day
0,2020-01-01,1,0,0.309625,2020-01-01,1
1,2020-01-01,1,1,0.325750,2020-01-01,1
2,2020-01-01,1,2,0.332875,2020-01-01,1
3,2020-01-01,1,3,0.324250,2020-01-01,1
4,2020-01-01,1,4,0.331750,2020-01-01,1


In [66]:
pd.set_option('display.max_columns',20)
train_x = train1[used_feat]
train_y = train1['pressure']
test_x = test1[used_feat]
print(train_x.shape, test_x.shape)
record = pd.DataFrame(columns = ['iterations','learning_rate','MSE'])
# -----------------------------------------------
scores = []


params = {'learning_rate': 0.1, 
        'boosting_type': 'gbdt', 
        'objective': 'regression_l1',
        'metric': 'mae',
        'min_child_samples': 46, 
        'min_child_weight': 0.01,
        'feature_fraction': 0.8, 
        'bagging_fraction': 0.8, 
        'bagging_freq': 2, 
        'num_leaves': 16, 
        'max_depth': 5, 
        'n_jobs': -1, 
        'seed': 2019, 
        'verbosity': -1, 
       }



oof_train = np.zeros(len(train_x)) # oof_train [0,0,0,0,0.....,0]
print(type(oof_train))
print(oof_train)
#print(oof_train[])
preds = np.zeros(len(test_x))
folds = 5
seeds = [2048, 1997]
s = 0
for i in [100]:
            for j in np.arange(0.001):
                for seed in seeds: 
                    kfold = KFold(n_splits=folds, shuffle=True, random_state=seed)
                    for fold, (trn_idx, val_idx) in enumerate(kfold.split(train_x, train_y)): # trn_idx为训练集 val_idx为验证集
                        print('fold ', fold + 1)
                        #print('这是val_idx:')
                        #print(type(val_idx))
                        #print(val_idx)
        
                        x_trn, y_trn, x_val, y_val = train_x.iloc[trn_idx], train_y.iloc[trn_idx], train_x.iloc[val_idx], train_y.iloc[val_idx]
                            #print('这是x trn：') # x_trn为x的训练集 y_trn为y的训练集； x_val为x的验证集 y_val为y的验证集
                            #print(type(x_trn))
                            #print(x_trn)
        #print('这是y trn：')
        #print(y_trn)
        #print('这是x val：')
        #print(x_val)
        #print('这是y val：')
        #print(y_val)
        
        
        
                        train_pool = Pool(x_trn, y_trn)
                        val_pool = Pool(x_val, y_val)
                        model = CatBoostRegressor(iterations= 5000, depth=2, learning_rate= 0.002, loss_function='RMSE')
                        model.fit(train_pool)
        #preds = model.predict(test_pool)


        #model = lgb.train(params, train_set, num_boost_round=500000,
                            #valid_sets=(train_set, val_set), early_stopping_rounds=50,
                            #verbose_eval=50)
        
                        oof_train[val_idx] += model.predict(x_val) / len(seeds) # numpy ndarray嵌套
        #preds += model.predict(test_x) / folds / len(seeds)
                        del x_trn, y_trn, x_val, y_val, model, train_pool, val_pool
                        gc.collect()
    
                        mse = (mean_squared_error(oof_train, train1['pressure']))
    
                        print('-'*120)
                        print(f'i={i},j={j}时的MSE：')
                        print('MSE ', round(mse, 5))
                        record.loc[s] = [i, j, mse]
                        s += 1
                        
                


#test1['pressure'] = preds + Mon_2_1_2019
print(record)
record.to_csv('/Users/liruizhi/Desktop/测试数据2.csv')

(22320, 3) (10080, 3)
<class 'numpy.ndarray'>
[0. 0. 0. ... 0. 0. 0.]
fold  1
0:	learn: 0.0665987	total: 3.9ms	remaining: 19.5s
1:	learn: 0.0665588	total: 7.36ms	remaining: 18.4s
2:	learn: 0.0665191	total: 11.8ms	remaining: 19.6s
3:	learn: 0.0664795	total: 14.6ms	remaining: 18.3s
4:	learn: 0.0664401	total: 17.8ms	remaining: 17.8s
5:	learn: 0.0664008	total: 21.4ms	remaining: 17.8s
6:	learn: 0.0663616	total: 25.8ms	remaining: 18.4s
7:	learn: 0.0663225	total: 28.8ms	remaining: 18s
8:	learn: 0.0662836	total: 31.7ms	remaining: 17.6s
9:	learn: 0.0662449	total: 35.3ms	remaining: 17.6s
10:	learn: 0.0662062	total: 38.8ms	remaining: 17.6s
11:	learn: 0.0661677	total: 42.9ms	remaining: 17.8s
12:	learn: 0.0661303	total: 48ms	remaining: 18.4s
13:	learn: 0.0660921	total: 49.8ms	remaining: 17.7s
14:	learn: 0.0660571	total: 51.1ms	remaining: 17s
15:	learn: 0.0660190	total: 52.9ms	remaining: 16.5s
16:	learn: 0.0659811	total: 54.8ms	remaining: 16.1s
17:	learn: 0.0659434	total: 56.1ms	remaining: 15.5s
18:

212:	learn: 0.0606017	total: 350ms	remaining: 7.87s
213:	learn: 0.0605820	total: 352ms	remaining: 7.87s
214:	learn: 0.0605639	total: 355ms	remaining: 7.9s
215:	learn: 0.0605447	total: 358ms	remaining: 7.92s
216:	learn: 0.0605256	total: 360ms	remaining: 7.93s
217:	learn: 0.0605065	total: 362ms	remaining: 7.94s
218:	learn: 0.0604873	total: 364ms	remaining: 7.95s
219:	learn: 0.0604680	total: 367ms	remaining: 7.97s
220:	learn: 0.0604491	total: 369ms	remaining: 7.98s
221:	learn: 0.0604303	total: 372ms	remaining: 8s
222:	learn: 0.0604112	total: 374ms	remaining: 8.01s
223:	learn: 0.0603922	total: 376ms	remaining: 8.02s
224:	learn: 0.0603734	total: 378ms	remaining: 8.01s
225:	learn: 0.0603546	total: 379ms	remaining: 8s
226:	learn: 0.0603358	total: 380ms	remaining: 7.99s
227:	learn: 0.0603170	total: 381ms	remaining: 7.98s
228:	learn: 0.0602983	total: 383ms	remaining: 7.97s
229:	learn: 0.0602797	total: 385ms	remaining: 7.98s
230:	learn: 0.0602611	total: 387ms	remaining: 7.98s
231:	learn: 0.06024

447:	learn: 0.0573397	total: 712ms	remaining: 7.23s
448:	learn: 0.0573283	total: 714ms	remaining: 7.24s
449:	learn: 0.0573170	total: 717ms	remaining: 7.25s
450:	learn: 0.0573072	total: 718ms	remaining: 7.24s
451:	learn: 0.0572974	total: 719ms	remaining: 7.24s
452:	learn: 0.0572877	total: 722ms	remaining: 7.24s
453:	learn: 0.0572765	total: 724ms	remaining: 7.24s
454:	learn: 0.0572637	total: 725ms	remaining: 7.24s
455:	learn: 0.0572510	total: 726ms	remaining: 7.24s
456:	learn: 0.0572413	total: 728ms	remaining: 7.24s
457:	learn: 0.0572315	total: 730ms	remaining: 7.24s
458:	learn: 0.0572218	total: 731ms	remaining: 7.23s
459:	learn: 0.0572120	total: 733ms	remaining: 7.23s
460:	learn: 0.0572027	total: 734ms	remaining: 7.22s
461:	learn: 0.0571931	total: 735ms	remaining: 7.22s
462:	learn: 0.0571835	total: 737ms	remaining: 7.22s
463:	learn: 0.0571740	total: 738ms	remaining: 7.21s
464:	learn: 0.0571614	total: 739ms	remaining: 7.21s
465:	learn: 0.0571518	total: 740ms	remaining: 7.2s
466:	learn: 0

654:	learn: 0.0554726	total: 1.07s	remaining: 7.1s
655:	learn: 0.0554661	total: 1.07s	remaining: 7.11s
656:	learn: 0.0554575	total: 1.07s	remaining: 7.11s
657:	learn: 0.0554489	total: 1.08s	remaining: 7.11s
658:	learn: 0.0554404	total: 1.08s	remaining: 7.11s
659:	learn: 0.0554339	total: 1.08s	remaining: 7.11s
660:	learn: 0.0554277	total: 1.08s	remaining: 7.1s
661:	learn: 0.0554192	total: 1.08s	remaining: 7.1s
662:	learn: 0.0554128	total: 1.08s	remaining: 7.1s
663:	learn: 0.0554068	total: 1.09s	remaining: 7.09s
664:	learn: 0.0553943	total: 1.09s	remaining: 7.09s
665:	learn: 0.0553858	total: 1.09s	remaining: 7.09s
666:	learn: 0.0553774	total: 1.09s	remaining: 7.09s
667:	learn: 0.0553710	total: 1.09s	remaining: 7.08s
668:	learn: 0.0553626	total: 1.09s	remaining: 7.08s
669:	learn: 0.0553562	total: 1.09s	remaining: 7.08s
670:	learn: 0.0553499	total: 1.1s	remaining: 7.08s
671:	learn: 0.0553437	total: 1.1s	remaining: 7.07s
672:	learn: 0.0553373	total: 1.1s	remaining: 7.07s
673:	learn: 0.05532

888:	learn: 0.0540349	total: 1.43s	remaining: 6.62s
889:	learn: 0.0540295	total: 1.43s	remaining: 6.63s
890:	learn: 0.0540255	total: 1.44s	remaining: 6.63s
891:	learn: 0.0540216	total: 1.44s	remaining: 6.63s
892:	learn: 0.0540178	total: 1.44s	remaining: 6.62s
893:	learn: 0.0540137	total: 1.44s	remaining: 6.63s
894:	learn: 0.0540067	total: 1.44s	remaining: 6.62s
895:	learn: 0.0539997	total: 1.45s	remaining: 6.62s
896:	learn: 0.0539943	total: 1.45s	remaining: 6.62s
897:	learn: 0.0539826	total: 1.45s	remaining: 6.62s
898:	learn: 0.0539772	total: 1.45s	remaining: 6.62s
899:	learn: 0.0539731	total: 1.45s	remaining: 6.61s
900:	learn: 0.0539678	total: 1.45s	remaining: 6.61s
901:	learn: 0.0539637	total: 1.45s	remaining: 6.61s
902:	learn: 0.0539596	total: 1.46s	remaining: 6.61s
903:	learn: 0.0539543	total: 1.46s	remaining: 6.61s
904:	learn: 0.0539426	total: 1.46s	remaining: 6.6s
905:	learn: 0.0539374	total: 1.46s	remaining: 6.6s
906:	learn: 0.0539321	total: 1.46s	remaining: 6.6s
907:	learn: 0.0

...

3966:	learn: 0.0454327	total: 8.41s	remaining: 2.19s
3967:	learn: 0.0454312	total: 8.41s	remaining: 2.19s
3968:	learn: 0.0454307	total: 8.42s	remaining: 2.19s
3969:	learn: 0.0454281	total: 8.42s	remaining: 2.18s
3970:	learn: 0.0454277	total: 8.42s	remaining: 2.18s
3971:	learn: 0.0454259	total: 8.42s	remaining: 2.18s
3972:	learn: 0.0454241	total: 8.42s	remaining: 2.18s
3973:	learn: 0.0454225	total: 8.43s	remaining: 2.17s
3974:	learn: 0.0454220	total: 8.43s	remaining: 2.17s
3975:	learn: 0.0454194	total: 8.43s	remaining: 2.17s
3976:	learn: 0.0454180	total: 8.43s	remaining: 2.17s
3977:	learn: 0.0454174	total: 8.43s	remaining: 2.17s
3978:	learn: 0.0454148	total: 8.43s	remaining: 2.16s
3979:	learn: 0.0454122	total: 8.43s	remaining: 2.16s
3980:	learn: 0.0454105	total: 8.44s	remaining: 2.16s
3981:	learn: 0.0454100	total: 8.44s	remaining: 2.16s
3982:	learn: 0.0454082	total: 8.44s	remaining: 2.15s
3983:	learn: 0.0454065	total: 8.44s	remaining: 2.15s
3984:	learn: 0.0454047	total: 8.44s	remaining:

4217:	learn: 0.0450049	total: 8.78s	remaining: 1.63s
4218:	learn: 0.0450033	total: 8.78s	remaining: 1.63s
4219:	learn: 0.0450029	total: 8.78s	remaining: 1.62s
4220:	learn: 0.0450025	total: 8.78s	remaining: 1.62s
4221:	learn: 0.0450020	total: 8.79s	remaining: 1.62s
4222:	learn: 0.0450010	total: 8.79s	remaining: 1.62s
4223:	learn: 0.0450005	total: 8.79s	remaining: 1.61s
4224:	learn: 0.0449994	total: 8.79s	remaining: 1.61s
4225:	learn: 0.0449977	total: 8.79s	remaining: 1.61s
4226:	learn: 0.0449971	total: 8.79s	remaining: 1.61s
4227:	learn: 0.0449967	total: 8.79s	remaining: 1.6s
4228:	learn: 0.0449963	total: 8.8s	remaining: 1.6s
4229:	learn: 0.0449910	total: 8.8s	remaining: 1.6s
4230:	learn: 0.0449857	total: 8.8s	remaining: 1.6s
4231:	learn: 0.0449832	total: 8.8s	remaining: 1.6s
4232:	learn: 0.0449779	total: 8.8s	remaining: 1.59s
4233:	learn: 0.0449774	total: 8.8s	remaining: 1.59s
4234:	learn: 0.0449761	total: 8.8s	remaining: 1.59s
4235:	learn: 0.0449736	total: 8.81s	remaining: 1.59s
4236:

4477:	learn: 0.0445968	total: 9.14s	remaining: 1.06s
4478:	learn: 0.0445959	total: 9.14s	remaining: 1.06s
4479:	learn: 0.0445935	total: 9.15s	remaining: 1.06s
4480:	learn: 0.0445927	total: 9.15s	remaining: 1.06s
4481:	learn: 0.0445918	total: 9.15s	remaining: 1.06s
4482:	learn: 0.0445908	total: 9.15s	remaining: 1.05s
4483:	learn: 0.0445900	total: 9.15s	remaining: 1.05s
4484:	learn: 0.0445892	total: 9.15s	remaining: 1.05s
4485:	learn: 0.0445882	total: 9.16s	remaining: 1.05s
4486:	learn: 0.0445877	total: 9.16s	remaining: 1.05s
4487:	learn: 0.0445872	total: 9.16s	remaining: 1.04s
4488:	learn: 0.0445864	total: 9.16s	remaining: 1.04s
4489:	learn: 0.0445859	total: 9.16s	remaining: 1.04s
4490:	learn: 0.0445851	total: 9.16s	remaining: 1.04s
4491:	learn: 0.0445846	total: 9.17s	remaining: 1.04s
4492:	learn: 0.0445832	total: 9.17s	remaining: 1.03s
4493:	learn: 0.0445824	total: 9.17s	remaining: 1.03s
4494:	learn: 0.0445775	total: 9.17s	remaining: 1.03s
4495:	learn: 0.0445760	total: 9.17s	remaining:

4732:	learn: 0.0442112	total: 9.51s	remaining: 537ms
4733:	learn: 0.0442108	total: 9.51s	remaining: 535ms
4734:	learn: 0.0442100	total: 9.51s	remaining: 533ms
4735:	learn: 0.0442096	total: 9.52s	remaining: 530ms
4736:	learn: 0.0442092	total: 9.52s	remaining: 528ms
4737:	learn: 0.0442087	total: 9.52s	remaining: 526ms
4738:	learn: 0.0442079	total: 9.52s	remaining: 524ms
4739:	learn: 0.0442035	total: 9.52s	remaining: 522ms
4740:	learn: 0.0442026	total: 9.52s	remaining: 520ms
4741:	learn: 0.0442014	total: 9.53s	remaining: 518ms
4742:	learn: 0.0442010	total: 9.53s	remaining: 516ms
4743:	learn: 0.0442006	total: 9.53s	remaining: 514ms
4744:	learn: 0.0441994	total: 9.53s	remaining: 512ms
4745:	learn: 0.0441971	total: 9.53s	remaining: 510ms
4746:	learn: 0.0441966	total: 9.53s	remaining: 508ms
4747:	learn: 0.0441943	total: 9.53s	remaining: 506ms
4748:	learn: 0.0441929	total: 9.54s	remaining: 504ms
4749:	learn: 0.0441924	total: 9.54s	remaining: 502ms
4750:	learn: 0.0441916	total: 9.54s	remaining:

4990:	learn: 0.0438572	total: 9.88s	remaining: 17.8ms
4991:	learn: 0.0438563	total: 9.88s	remaining: 15.8ms
4992:	learn: 0.0438551	total: 9.88s	remaining: 13.8ms
4993:	learn: 0.0438547	total: 9.88s	remaining: 11.9ms
4994:	learn: 0.0438526	total: 9.88s	remaining: 9.89ms
4995:	learn: 0.0438522	total: 9.88s	remaining: 7.91ms
4996:	learn: 0.0438514	total: 9.88s	remaining: 5.93ms
4997:	learn: 0.0438504	total: 9.89s	remaining: 3.96ms
4998:	learn: 0.0438493	total: 9.89s	remaining: 1.98ms
4999:	learn: 0.0438471	total: 9.89s	remaining: 0us
------------------------------------------------------------------------------------------------------------------------
i=100,j=0.0时的MSE：
MSE  0.00186
   iterations  learning_rate       MSE
0       100.0            0.0  0.070720
1       100.0            0.0  0.058636
2       100.0            0.0  0.046526
3       100.0            0.0  0.034409
4       100.0            0.0  0.022317
5       100.0            0.0  0.018205
6       100.0            0.0  0.014135

In [ ]:
train1.head()